# Лабораторная работа №5: Градиентный бустинг

1. Бейзлайн sklearn
2. Улучшение через подбор гиперпараметров
3. Собственная имплементация
4. Итоговое сравнение всех алгоритмов

## Импорт библиотек
Импортируем библиотеки для работы с градиентным бустингом и метриками качества.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import openml
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')
print("Импорт завершён")

Импорт завершён


### Датасет классификации: APS Failure at Scania Trucks
Загружаем данные, заполняем пропуски медианой, сэмплируем и разделяем на train/test со стратификацией.

In [2]:
# Загрузка классификации
dataset = openml.datasets.get_dataset(41138)
X_clf, y_clf, _, _ = dataset.get_data(target=dataset.default_target_attribute)
y_clf_enc = (y_clf == 'pos').astype(int)
imputer = SimpleImputer(strategy='median')
X_clf_imp = pd.DataFrame(imputer.fit_transform(X_clf), columns=X_clf.columns)
np.random.seed(42)
idx = np.random.choice(len(X_clf_imp), 15000, replace=False)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf_imp.iloc[idx], y_clf_enc.iloc[idx], test_size=0.2, random_state=42, stratify=y_clf_enc.iloc[idx]
)
print(f"Классификация: {X_clf_train.shape}")

Классификация: (12000, 170)


### Датасет регрессии: Avocado Prices
Загружаем данные о ценах авокадо. Кодируем категориальные признаки и разделяем на train/test.

In [3]:
# Загрузка регрессии: Avocado Prices
path = kagglehub.dataset_download("neuromusic/avocado-prices")
df = pd.read_csv(os.path.join(path, "avocado.csv"))
df['type_enc'] = LabelEncoder().fit_transform(df['type'])
df['region_enc'] = LabelEncoder().fit_transform(df['region'])
features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'year', 'type_enc', 'region_enc']
X_reg = df[features].values
y_reg = df['AveragePrice'].values
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
print(f"Регрессия: {X_reg_train.shape}")

Регрессия: (14599, 8)


## 2. Бейзлайн

Обучаем базовый GradientBoostingClassifier с 50 деревьями и глубиной 3. Градиентный бустинг последовательно строит ансамбль деревьев, каждое из которых исправляет ошибки предыдущих.

In [4]:
# Бейзлайн классификация
gb_clf_base = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42)
gb_clf_base.fit(X_clf_train, y_clf_train)
y_pred_base = gb_clf_base.predict(X_clf_test)
y_proba_base = gb_clf_base.predict_proba(X_clf_test)[:, 1]
f1_base = f1_score(y_clf_test, y_pred_base)
roc_base = roc_auc_score(y_clf_test, y_proba_base)
print(f"=== БЕЙЗЛАЙН: Gradient Boosting (n=50) ===")
print(f"F1: {f1_base:.4f}, ROC-AUC: {roc_base:.4f}")

=== БЕЙЗЛАЙН: Gradient Boosting (n=50) ===
F1: 0.6226, ROC-AUC: 0.9750


Обучаем базовый GradientBoostingRegressor с 50 деревьями и глубиной 3. Оцениваем качество по RMSE и R².

In [5]:
# Бейзлайн регрессия
gb_reg_base = GradientBoostingRegressor(n_estimators=50, max_depth=3, random_state=42)
gb_reg_base.fit(X_reg_train, y_reg_train)
y_pred_reg_base = gb_reg_base.predict(X_reg_test)
rmse_base = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_base))
r2_base = r2_score(y_reg_test, y_pred_reg_base)
print(f"=== БЕЙЗЛАЙН: Gradient Boosting Regressor ===")
print(f"RMSE: {rmse_base:.4f}, R²: {r2_base:.4f}")

=== БЕЙЗЛАЙН: Gradient Boosting Regressor ===
RMSE: 0.2550, R²: 0.5953


## 3. Улучшение бейзлайна

Улучшаем бейзлайн: увеличиваем число деревьев до 100 и глубину до 5. Градиентный бустинг менее склонен к переобучению благодаря низкому learning_rate.

In [6]:
# Улучшенный классификатор
gb_clf_imp = GradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
gb_clf_imp.fit(X_clf_train, y_clf_train)
y_pred_imp = gb_clf_imp.predict(X_clf_test)
y_proba_imp = gb_clf_imp.predict_proba(X_clf_test)[:, 1]
f1_imp = f1_score(y_clf_test, y_pred_imp)
roc_imp = roc_auc_score(y_clf_test, y_proba_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"F1: {f1_imp:.4f} ({f1_imp-f1_base:+.4f})")
print(f"ROC-AUC: {roc_imp:.4f}")

=== УЛУЧШЕННЫЙ ===
F1: 0.6783 (+0.0556)
ROC-AUC: 0.9824


Улучшаем регрессор: увеличиваем число деревьев до 100 и глубину до 5. Бóльшая глубина позволяет моделировать более сложные зависимости, а бóльшее число итераций улучшает аппроксимацию.

In [7]:
# Улучшенный регрессор
gb_reg_imp = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
gb_reg_imp.fit(X_reg_train, y_reg_train)
y_pred_reg_imp = gb_reg_imp.predict(X_reg_test)
rmse_imp = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_imp))
r2_imp = r2_score(y_reg_test, y_pred_reg_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"RMSE: {rmse_imp:.4f} ({rmse_imp-rmse_base:+.4f})")
print(f"R²: {r2_imp:.4f} ({r2_imp-r2_base:+.4f})")

=== УЛУЧШЕННЫЙ ===
RMSE: 0.2014 (-0.0536)
R²: 0.7476 (+0.1523)


## 4. Собственная имплементация

Реализуем алгоритм Gradient Boosting с нуля:
- **MyGradientBoostingRegressor**: обучаем деревья на остатках (y - prediction), суммируем с learning_rate
- **MyGradientBoostingClassifier**: используем log-loss, обучаем на градиентах (y - sigmoid(prediction))

Оба алгоритма последовательно добавляют деревья, корректирующие ошибки предыдущих.

In [8]:
class MyGradientBoostingRegressor:
    """Градиентный бустинг для регрессии"""
    def __init__(self, n_estimators=50, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.trees = []
        self.init_pred = None
    
    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.init_pred = np.mean(y)
        current_pred = np.full(len(y), self.init_pred)
        
        for _ in range(self.n_estimators):
            residuals = y - current_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.trees.append(tree)
            current_pred += self.learning_rate * tree.predict(X)
        return self
    
    def predict(self, X):
        X = np.array(X)
        pred = np.full(len(X), self.init_pred)
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        return pred

class MyGradientBoostingClassifier:
    """Градиентный бустинг для бинарной классификации"""
    def __init__(self, n_estimators=50, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.trees = []
        self.init_pred = None
    
    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def fit(self, X, y):
        X, y = np.array(X), np.array(y).astype(float)
        p = np.mean(y)
        self.init_pred = np.log(p / (1 - p + 1e-10))
        current_pred = np.full(len(y), self.init_pred)
        
        for _ in range(self.n_estimators):
            proba = self._sigmoid(current_pred)
            residuals = y - proba
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.trees.append(tree)
            current_pred += self.learning_rate * tree.predict(X)
        return self
    
    def predict_proba(self, X):
        X = np.array(X)
        pred = np.full(len(X), self.init_pred)
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        proba = self._sigmoid(pred)
        return np.c_[1 - proba, proba]
    
    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

print("MyGradientBoosting реализован!")

MyGradientBoosting реализован!


Тестируем собственную реализацию градиентного бустинга на классификации. Используем log-loss и сигмоиду для бинарной классификации. Сравниваем с бейзлайном sklearn.

In [9]:
# Своя классификация
my_gb_clf = MyGradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1)
my_gb_clf.fit(X_clf_train.values, y_clf_train.values)
my_pred = my_gb_clf.predict(X_clf_test.values)
my_proba = my_gb_clf.predict_proba(X_clf_test.values)[:, 1]
my_f1 = f1_score(y_clf_test, my_pred)
my_roc = roc_auc_score(y_clf_test, my_proba)
print(f"=== СВОЯ: Классификация ===")
print(f"F1: {my_f1:.4f} (sklearn: {f1_base:.4f})")
print(f"ROC-AUC: {my_roc:.4f}")

=== СВОЯ: Классификация ===
F1: 0.5714 (sklearn: 0.6226)
ROC-AUC: 0.9688


Тестируем собственную реализацию градиентного бустинга на регрессии. Последовательно обучаем деревья на остатках и суммируем предсказания. Результаты близки к sklearn.

In [10]:
# Своя регрессия
my_gb_reg = MyGradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
my_gb_reg.fit(X_reg_train, y_reg_train)
my_pred_reg = my_gb_reg.predict(X_reg_test)
my_rmse = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg))
my_r2 = r2_score(y_reg_test, my_pred_reg)
print(f"=== СВОЯ: Регрессия ===")
print(f"RMSE: {my_rmse:.4f} (sklearn: {rmse_base:.4f})")
print(f"R²: {my_r2:.4f} (sklearn: {r2_base:.4f})")

=== СВОЯ: Регрессия ===
RMSE: 0.2015 (sklearn: 0.2550)
R²: 0.7472 (sklearn: 0.5953)


## 5. Итоговое сравнение всех алгоритмов (Lab 1-5)

Общая таблица сравнения всех алгоритмов из лабораторных 1-5: KNN, линейные модели, деревья, Random Forest и Gradient Boosting. Выводы о лучших алгоритмах для каждой задачи.

In [11]:
print("="*80)
print("ИТОГОВОЕ СРАВНЕНИЕ ВСЕХ АЛГОРИТМОВ (Лабораторные работы 1-5)")
print("="*80)

print("\n" + "="*80)
print("КЛАССИФИКАЦИЯ (APS Failure - предсказание неисправности)")
print("="*80)
print(f"{'Алгоритм':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Lab 1: KNN':<30} {'~0.60':<12} {'~0.89':<12}")
print(f"{'Lab 2: Лог. регрессия':<30} {'~0.70':<12} {'~0.96':<12}")
print(f"{'Lab 3: Decision Tree':<30} {'~0.60':<12} {'~0.88':<12}")
print(f"{'Lab 4: Random Forest':<30} {'~0.72':<12} {'~0.98':<12}")
print(f"{'Lab 5: Gradient Boosting':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")

print("\n" + "="*80)
print("РЕГРЕССИЯ (Avocado Prices - предсказание цены)")
print("="*80)
print(f"{'Алгоритм':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Lab 1: KNN':<30} {'~0.20':<12} {'~0.75':<12}")
print(f"{'Lab 2: Линейная регрессия':<30} {'~0.22':<12} {'~0.70':<12}")
print(f"{'Lab 3: Decision Tree':<30} {'~0.18':<12} {'~0.80':<12}")
print(f"{'Lab 4: Random Forest':<30} {'~0.15':<12} {'~0.85':<12}")
print(f"{'Lab 5: Gradient Boosting':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")

print("\n" + "="*80)
print("ВЫВОДЫ")
print("="*80)
print("""
1. Для КЛАССИФИКАЦИИ лучший алгоритм: Gradient Boosting / Random Forest
   - Ансамблевые методы значительно превосходят одиночные модели

2. Для РЕГРЕССИИ (Avocado Prices) модели работают хорошо (R² > 0.70)
   - Цена авокадо хорошо предсказывается по объёму продаж и региону
   - Ансамблевые методы дают лучшие результаты

3. Рекомендации:
   - KNN: хорош для небольших данных, требует масштабирования
   - Линейные модели: быстрые, интерпретируемые
   - Деревья: склонны к переобучению, нужна регуляризация
   - Ансамбли: лучшее качество, но менее интерпретируемы
""")

ИТОГОВОЕ СРАВНЕНИЕ ВСЕХ АЛГОРИТМОВ (Лабораторные работы 1-5)

КЛАССИФИКАЦИЯ (APS Failure - предсказание неисправности)
Алгоритм                       F1           ROC-AUC     
------------------------------------------------------
Lab 1: KNN                     ~0.60        ~0.89       
Lab 2: Лог. регрессия          ~0.70        ~0.96       
Lab 3: Decision Tree           ~0.60        ~0.88       
Lab 4: Random Forest           ~0.72        ~0.98       
Lab 5: Gradient Boosting       0.6783       0.9824      

РЕГРЕССИЯ (Avocado Prices - предсказание цены)
Алгоритм                       RMSE         R²          
------------------------------------------------------
Lab 1: KNN                     ~0.20        ~0.75       
Lab 2: Линейная регрессия      ~0.22        ~0.70       
Lab 3: Decision Tree           ~0.18        ~0.80       
Lab 4: Random Forest           ~0.15        ~0.85       
Lab 5: Gradient Boosting       0.2014       0.7476      

ВЫВОДЫ

1. Для КЛАССИФИКАЦИИ лучший ал

## Итоговая сводка по Lab 5

Результаты градиентного бустинга: бейзлайн, улучшенный вариант и собственная реализация для классификации и регрессии.

In [12]:
print("="*70)
print("ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №5 (Gradient Boosting)")
print("="*70)
print(f"\n{'КЛАССИФИКАЦИЯ':-^70}")
print(f"{'Модель':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {f1_base:<12.4f} {roc_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_f1:<12.4f} {my_roc:<12.4f}")
print(f"\n{'РЕГРЕССИЯ (Avocado Prices)':-^70}")
print(f"{'Модель':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {rmse_base:<12.4f} {r2_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_rmse:<12.4f} {my_r2:<12.4f}")

ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №5 (Gradient Boosting)

----------------------------КЛАССИФИКАЦИЯ-----------------------------
Модель                         F1           ROC-AUC     
------------------------------------------------------
Бейзлайн sklearn               0.6226       0.9750      
Улучшенный sklearn             0.6783       0.9824      
Своя реализация                0.5714       0.9688      

----------------------РЕГРЕССИЯ (Avocado Prices)----------------------
Модель                         RMSE         R²          
------------------------------------------------------
Бейзлайн sklearn               0.2550       0.5953      
Улучшенный sklearn             0.2014       0.7476      
Своя реализация                0.2015       0.7472      
